In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium
pip install pathos

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
from pathos.multiprocessing import ProcessingPool
from pathos.multiprocessing import ProcessPool
import os
import progressbar
from more_itertools import chunked

In [ ]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

KeyboardInterrupt: ignored

#### 1. 크롤링

In [ ]:
# 크롤링할 페이지 URL
def Channel_crawling(url, temp_df, driver):
    channel_num = int(url[40:])
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    Channel_name = str(soup.select('strong.ProfileCreator_name_rQYEk'))
    Channel_name = (Channel_name.split('<')[1])
    Channel_name = (Channel_name.split('>')[1])
    temp_df.loc[channel_num, ['Channel_Name']]= Channel_name

    temp_df.loc[channel_num, ['Channel_Number']] = channel_num


    Num_sub = str(soup.select('div.ProfileCreator_fan_lUrMY'))
    Num_sub = (Num_sub.split('<')[1])
    Num_sub = (Num_sub.split('>')[1])
    Num_sub = (Num_sub.split('명')[0])
    
    if '만' in Num_sub:
        Num_sub = (Num_sub.split('만')[0])
        Num_sub = float(Num_sub) * 10000
        Num_sub = int(Num_sub)
    else:
        Num_sub = Num_sub.replace(',', '')
        Num_sub = int(Num_sub)
        
    temp_df.loc[channel_num, ['Num_Sub']]= Num_sub

    Num_Video = str(soup.select('div.ItemCount_wrap_dYzmP'))
    try:
        Num_Video = (Num_Video.split('<')[1])
        Num_Video = (Num_Video.split('>')[1])
        Num_Video = (Num_Video.split(' ')[1])
        temp_df.loc[channel_num, ['Num_Video']]= Num_Video
        
    except:
        temp_df.loc[channel_num, ['Num_Video']]= 0
    
    View_Video = soup.select('span.VideoViewCount_count_CKLvb')
    view_list = []
    try:
        for i in View_Video:
            i = str(i)
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            i = (i.split(' ')[0])
            
            if '만' in i:
                i = (i.split('만')[0])
                i = float(i) * 10000
                i = int(i)
            else:
                i = i.replace(',', '')
                i = int(i)
                
            view_list.append(i)

        num_list = len(view_list)
        temp_df.loc[channel_num, 'Total_View']= sum(view_list)
        temp_df.loc[channel_num, 'Mean_View']= (sum(view_list) / num_list)
        temp_df.loc[channel_num, 'Best_View']= max(view_list)
        temp_df.loc[channel_num, 'Worst_View']= min(view_list)
        temp_df.loc[channel_num, 'X_num_video']= num_list

    except:
        temp_df.loc[channel_num, 'Total_View']= 0
        temp_df.loc[channel_num, 'Best_View']= 0
        temp_df.loc[channel_num, 'Worst_View']= 0

In [ ]:
empty = []
temp_df = pd.DataFrame({'Channel_Name' : empty, 'Chaneel_Number' : empty, 'Num_Sub' : empty, 'Num_Video' : empty, 'Total_View' : empty, 'Mean_View' : empty, 'Best_View' : empty, 'Worst_View' : empty, 'X_num_video' : empty})
total_num = 15

bar = progressbar.ProgressBar(maxval=(total_num)).start()

for l, j in enumerate(range(total_num)):
    print(j)
    bar.update(l)
    k = j+1
    url = f'https://shoppinglive.naver.com/channels/{k}?sort=VIEW_COUNT'
    try:
        Channel_crawling(url, temp_df, driver)
    except:
        continue
    
bar.finish()
temp_df.to_csv('/content/drive/MyDrive/LiveCommerse/Channel_Rank_3.csv')

In [ ]:
temp_df.head(50)

,Channel_Name,Chaneel_Number,Num_Sub,Num_Video,Total_View,Mean_View,Best_View,Worst_View,X_num_video
2,산토리니,NaN,83.0,48,116.0,5.800000,47.0,0.0,20.0
3,아디다스,NaN,21000.0,15,173516.0,11567.733333,100000.0,3426.0,15.0
4,구구구,NaN,36.0,38,138.0,6.900000,27.0,0.0,20.0
5,뷰티테스트1_인디,NaN,2.0,18,218.0,12.111111,47.0,2.0,18.0
6,리빙테스트_홈앤데코,NaN,0.0,0,0.0,NaN,0.0,0.0,NaN
7,롯데몰 진주점 토박스,NaN,38.0,1,32.0,32.000000,32.0,32.0,1.0
8,_thesext_,NaN,267.0,8,77736.0,9717.000000,23000.0,381.0,8.0
9,별,NaN,40.0,2,481.0,240.500000,459.0,22.0,2.0
10,nnino,NaN,2112.0,8,180048.0,22506.000000,61000.0,2048.0,8.0
11,웰모아 공식스토어,NaN,5317.0,77,150250.0,7512.500000,63000.0,834.0,20.0


In [ ]:
temp_df.index

Index(['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '27',
       '28', '29', '30'],
      dtype='object')

In [ ]:
missing_index = list(set(range(temp_df.index.min(), temp_df.index.max()+1)) - set(temp_df.index))
print(missing_index)

[]


In [ ]:
mpty = []
temp_df = pd.DataFrame({'Channel_Name' : empty, 'Num_Sub' : empty, 'Num_Video' : empty, 'Total_View' : empty, 'Best_View' : empty, 'Worst_View' : empty})

[<span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>,
 <span class="VideoViewCount_count_CKLvb">0 시청</span>]

In [ ]:
min(view_list)

1808

In [ ]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 64165 to 64165
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Channel_Name  1 non-null      object 
 1   Num_Sub       1 non-null      object 
 2   Num_Video     1 non-null      object 
 3   Total_View    1 non-null      float64
 4   Best_View     1 non-null      float64
 5   Worst_View    1 non-null      float64
dtypes: float64(3), object(3)
memory usage: 164.0+ bytes


In [ ]:
profile_creator_fan

[<div data-reactroot="" id="__next"></div>]

In [ ]:
# 특정 사이트 크롤링
url= 'https://shoppinglive.naver.com/channels/64165'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
temp = soup.select('div.ProfileCreator_fan_lUrMY')
temp


[]

In [ ]:
temp

[]

In [ ]:
print(temp)

None


In [ ]:
# 뷰티 인기순 크롤링
def URL_Crwaler(url, path):
    driver = webdriver.Chrome(chrome_driver_path)
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list_video = soup.find('div', class_= 'ProfileCreator_fan_lUrMY').find_all('a')
    list_video_top10 = list_video[:20]

    top_10_live_list = []
    top_10_channel_list = []

    for i in list_video_top10:
        a = str(i['href'])
        if 'channels' in a:
            a = 'https://shoppinglive.naver.com' + a
            top_10_channel_list.append(a)
        else:
            top_10_live_list.append(a)

    temp_df = pd.DataFrame({'Video_url' : top_10_live_list, 'Channel_url' : top_10_channel_list})
    temp_df.to_csv(path, index=False)

In [ ]:
Video_save_path = 'D:\Google Drive\내 드라이브\LiveCommerse\Video_df'
url_beauty = 'https://shoppinglive.naver.com/categories/dc:1?sort=VIEWER'
url_food = 'https://shoppinglive.naver.com/categories/dc:2?sort=VIEWER'
url_fashion = 'https://shoppinglive.naver.com/categories/dc:3?sort=VIEWER'
url_life = 'https://shoppinglive.naver.com/categories/dc:4?sort=VIEWER'
url_tour = 'https://shoppinglive.naver.com/categories/dc:5?sort=VIEWER'
url_kids = 'https://shoppinglive.naver.com/categories/dc:6?sort=VIEWER'
url_tech = 'https://shoppinglive.naver.com/categories/dc:7?sort=VIEWER'
url_hobby = 'https://shoppinglive.naver.com/categories/dc:8?sort=VIEWER'
url_culture = 'https://shoppinglive.naver.com/categories/dc:9?sort=VIEWER'

In [ ]:
URL_Crwaler(url_beauty, (Video_save_path+'\\beauty.csv'))
URL_Crwaler(url_food, (Video_save_path+'\\food.csv'))
URL_Crwaler(url_fashion, (Video_save_path+'\\fashion.csv'))
URL_Crwaler(url_life, (Video_save_path+'\\life.csv'))
URL_Crwaler(url_tour, (Video_save_path+'\\tour.csv'))
URL_Crwaler(url_kids, (Video_save_path+'\\kids.csv'))
URL_Crwaler(url_tech, (Video_save_path+'\\tech.csv'))
URL_Crwaler(url_hobby, (Video_save_path+'\\hobby.csv'))

In [ ]:
# 크롤러 생성

class MyCrawler:
    def __init__(self, numprocess, webdriverpath, path_save):
        self.n_process = numprocess
        self.pool = ProcessPool(nodes=self.n_process)
        self.webdriverpath = webdriverpath
        self.path_save = path_save
        
    def multi_processing(self, sites, repeat):
        def chat_crawler(url):
            # 사용하는 Library Import
            from selenium import webdriver
            from selenium.webdriver.common.by import By
            import time
            import pandas as pd
            from bs4 import BeautifulSoup

            # 원하는 사이트 로드
            driver = webdriver.Chrome(self.webdriverpath)
            driver.get(url)
            save_number = str(url)
            save_number = save_number[44:50]
            
            # 1분 대기 ... 여러 사이트 열어서 시간이 좀 걸림.
            time.sleep(6*self.n_process)

            # 자동으로 2배속 설정.
            buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
            buttun.click()
            time.sleep(1)
            buttun.click()
            
            # 반복 횟수 설정.
            repeat_n = repeat
            empty = []
            df = pd.DataFrame({'name' : empty, 'chat' : empty})
            
            # 채팅 및 아이디 크롤링.
            for re_n in range(repeat_n):
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
            
                chat_list_temp = []
                for i in temp_list:
                    for j in i:
                        for k in j:
                            chat = k.select('span')
                            chat = str(chat)
                            chat_list_temp.append(chat)

                name_list_temp = []
                for i in temp_list:
                    for j in i:
                        name = j.select('strong')
                        name = str(name)
                        if 'nickname' in name:
                            name_list_temp.append(name)

                chat_list = []
                for i in chat_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        chat_list.append(i)
                    except:
                        continue
                    
                name_list = []
                for i in name_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        name_list.append(i)
                    except:
                        continue
                df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
                df = pd.concat([df, df_temp], ignore_index=True)
                
                time.sleep(150)

            # df 형성 및 저장.
            df.drop_duplicates(inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.to_csv(self.path_save + '\{}.csv' .format(save_number), index=False)
            
            driver.quit()
        
        self.pool.map(chat_crawler, sites)

In [ ]:
df_list = os.listdir('D:\Google Drive\내 드라이브\LiveCommerse\Video_df')

for i in df_list:
    df = pd.read_csv('D:\Google Drive\내 드라이브\LiveCommerse\Video_df'+f'\{i}')
    url_list = list(df.Video_url)
    
    url_list_1 = url_list[:5]
    url_list_2 = url_list[5:]
    
    category_name = i[:-4]
    path = f'D:\Google Drive\내 드라이브\LiveCommerse\df_save\saveforder_{category_name}'
    
    try:
        Crawler1 = MyCrawler(len(url_list_1), chrome_driver_path, path)
    except:
        continue
    
    try:
        Crawler2 = MyCrawler(len(url_list_2), chrome_driver_path, path)
    except:
        continue
    
    try:
        Crawler1.multi_processing(url_list_1, 24)
    except:
        continue
    
    try:
        Crawler2.multi_processing(url_list_2, 24)
    except:
        continue

In [ ]:
# 크롤러 할당

num1 = MyCrawler(len(crawlnum1), chrome_driver_path, save_path_iope)
num2 = MyCrawler(len(crawlnum2), chrome_driver_path, save_path_iope)

In [ ]:
# 크롤러 병렬 실행

num1.multi_processing(crawlnum1, 12)
num2.multi_processing(crawlnum2, 12)

In [ ]:
# csv 파일 전체 통합

empty = []
df = pd.DataFrame({'name' : empty, 'chat' : empty})

data_csv = os.listdir(save_path_iope)

for i in data_csv:
    df_temp = pd.read_csv(save_path_iope + f'\{i}')
    df = pd.concat([df, df_temp], ignore_index=True)
    
df.reset_index(drop=True, inplace=True)
df.to_csv(save_path_iope + '\\total.csv', index=False)


In [ ]:
df = pd.read_csv(r'G:\내 드라이브\LiveCommerse\saveforder_iope\total.csv')
df.name.nunique()

4520

In [ ]:
# 전체 df 가져와서 구매번호 인증 관련된 채팅 제거.

df = pd.read_csv(r'G:\내 드라이브\LiveCommerse\saveforder_iope\total.csv')

pattern = '\d{16,}'
drop_df = df[~(df['chat'].str.contains(pattern))]
drop_df.reset_index(drop=True, inplace=True)
drop_df

,name,chat
0,김따뜻,기대기대🤗
1,져니,안녕하세요
2,eundoki2004,안녕하세요~ㅎㅎ
3,져니,반가워요
4,져니,잘볼게요
...,...,...
10513,wls,당첨되고 시프요
10514,딸기토끼,대박적혜택! 아까 더살껄...
10515,인문논술독서논술과외,헐50종
10516,젤리주주,카페인샷 사은품으로 가는 포텐셜크림이랑 꼭 함께 바르세요. 시너지 폭발이예요!!


In [ ]:
drop_df.name.nunique()

4183

##### 아래는 연습장임

In [ ]:
# 크롤러 만들기.
def scrape(url):
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
    driver.get(url)

In [ ]:
# 자동 2배속

def sppedx2(driver):
    buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
    buttun.click()
    time.sleep(1)
    buttun.click()

In [ ]:
# 자동 크롤링 반복 후 df 형성

def crawler(repeat, savepath):
    repeat_n = repeat
    empty = []
    df = pd.DataFrame({'name' : empty, 'chat' : empty})
    
    for re_n in range(repeat_n):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
        chat_list_temp = []
        for i in temp_list:
            for j in i:
                for k in j:
                    chat = k.select('span')
                    chat = str(chat)
                    chat_list_temp.append(chat)

        name_list_temp = []
        for i in temp_list:
            for j in i:
                name = j.select('strong')
                name = str(name)
                if 'nickname' in name:
                    name_list_temp.append(name)

        chat_list = []
        for i in chat_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                chat_list.append(i)
            except:
                continue
            
        name_list = []
        for i in name_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                name_list.append(i)
            except:
                continue
        df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
        df = pd.concat([df, df_temp], ignore_index=True)
        
        time.sleep(150)

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(savepath)

In [ ]:
driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
driver.get(replay_video_list[0])

time.sleep(5)
time.sleep(10)

# 자동 2배속
buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
buttun.click()
time.sleep(1)
buttun.click()


repeat = 12

empty = []
df = pd.DataFrame({'name': empty, 'chat' : empty})

for re_n in range(repeat):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
    chat_list_temp = []
    for i in temp_list:
        for j in i:
            for k in j:
                chat = k.select('span')
                chat = str(chat)
                chat_list_temp.append(chat)

    name_list_temp = []
    for i in temp_list:
        for j in i:
            name = j.select('strong')
            name = str(name)
            if 'nickname' in name:
                name_list_temp.append(name)

    chat_list = []
    for i in chat_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            chat_list.append(i)
        except:
            continue
        
    name_list = []
    for i in name_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            name_list.append(i)
        except:
            continue
    
    df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
    df = pd.concat([df, df_temp], ignore_index=True)
    
    print(re_n)
    time.sleep(150)

df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/df_save_{}.csv' .format(url_num))

C:\Users\gkfua\AppData\Local\Temp\ipykernel_7904\3999771474.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')


In [ ]:
def scrape(url):
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    
    time.sleep(5)
    time.sleep(10)

    # 자동 2배속
    buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
    buttun.click()
    time.sleep(1)
    buttun.click()
    
    time.sleep(10)

In [ ]:
pool = ProcessPool(nodes=10)
pool.map(scrape, replay_video_list)

[None, None, None, None, None, None, None, None, None, None]

In [ ]:
class WebCrawler:
    def __init__(self):
        self.pool = ProcessPool(nodes=10)
        
    def open_browser(url):
        from selenium import webdriver
        driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
        driver.get(url)
        
    def multi_processing(self, url_list):
        self.pool.map(open_browser, url_list)


In [ ]:
len(replay_video_list)

10

In [ ]:
replay_video_list

['https://view.shoppinglive.naver.com/replays/869256?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/820001?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/809363?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/768444?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/719762?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/687769?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/646838?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/621921?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/620591?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/595816?fm=shoppinglive&sn=home']

In [ ]:
# 원하는 사이트 오픈

url_num = 2
driver = webdriver.Chrome(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/chromedriver_mac64/chromedriver')
driver.get(url_list[url_num])

C:\Users\gkfua\AppData\Local\Temp\ipykernel_19496\3999835344.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/chromedriver_mac64/chromedriver')


NameError: name 'url_list' is not defined

In [ ]:
repeat = 12

empty = []
df = pd.DataFrame({'name': empty, 'chat' : empty})

for re_n in range(repeat):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
    chat_list_temp = []
    for i in temp_list:
        for j in i:
            for k in j:
                chat = k.select('span')
                chat = str(chat)
                chat_list_temp.append(chat)

    name_list_temp = []
    for i in temp_list:
        for j in i:
            name = j.select('strong')
            name = str(name)
            if 'nickname' in name:
                name_list_temp.append(name)

    chat_list = []
    for i in chat_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            chat_list.append(i)
        except:
            continue
        
    name_list = []
    for i in name_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            name_list.append(i)
        except:
            continue
    
    df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
    df = pd.concat([df, df_temp], ignore_index=True)
    
    print(re_n)
    time.sleep(150)

df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/df_save_{}.csv' .format(url_num))

0
1
2
3
4
5
6
7
8
9
10
11
